In [ ]:
# ==============================================================================
# 🦅 CLOUDFOCUS: "Fusion-Lite" Training (FER2013 + RAF-DB Merged)
# ==============================================================================

# 1️⃣ Setup Environment
!pip install -q kaggle

import os
import glob
import shutil
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from google.colab import drive, files
from sklearn.utils import class_weight

# ⚠️ FORCE FLOAT32: Prevents "Flex Delegate" errors on Raspberry Pi
tf.keras.mixed_precision.set_global_policy('float32')
print("✅ Policy set to 'float32' (Safe Mode).")

# 2️⃣ Mount Drive
drive.mount('/content/drive')
CHECKPOINT_DIR = "/content/drive/MyDrive/FER_Fusion_RAFDB/"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# 3️⃣ Download NEW Dataset (FER2013 + RAF-DB Preprocessed)
if not os.path.exists("/root/.kaggle/kaggle.json"):
    print("\n🔑 Upload kaggle.json:")
    uploaded = files.upload()
    !mkdir -p /root/.kaggle
    !mv kaggle.json /root/.kaggle/
    !chmod 600 /root/.kaggle/kaggle.json

DATASET_PATH = "/content/fer_rafdb"
if not os.path.exists(DATASET_PATH):
    print("⬇️ Downloading Merged Dataset...")
    !kaggle datasets download -d fahadullaha/facial-emotion-recognition-dataset -p {DATASET_PATH} --unzip
    print("✅ Download Complete.")

# Locate the correct folder (The zip might extract into a subfolder)
TRAIN_DIR = DATASET_PATH
for root, dirs, files in os.walk(DATASET_PATH):
    if "happy" in dirs and "angry" in dirs:
        TRAIN_DIR = root
        break
print(f"📂 Dataset Root: {TRAIN_DIR}")

# 4️⃣ Pipeline Config (224x224 is standard for these models)
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# 5️⃣ Data Loading with Augmentation
# We split the single folder into Train (80%) and Validation (20%)
print("\n🔄 Creating Data Pipelines...")

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Augmentation (Crucial for "Wild" datasets)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomBrightness(0.1),
    layers.RandomContrast(0.1),
])

# Apply augmentation
train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

# Get Class Names
class_names = train_ds.class_names
print(f"🏷️ Classes: {class_names}")

# 6️⃣ The "Fusion-Lite" Architecture
# Fuses MobileNetV2 (Robust) + MobileNetV3-Small (Efficient)
def build_fusion_model():
    # Input: Raw RGB 0-255 (Standard for Pi)
    inputs = layers.Input(shape=(224, 224, 3), dtype=tf.float32)
    
    # --- Branch 1: MobileNetV2 (Feature Extraction) ---
    # Expects [-1, 1]
    x1_norm = layers.Rescaling(1./127.5, offset=-1)(inputs)
    base_v2 = applications.MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
    base_v2._name = "mobilenet_v2"
    base_v2.trainable = False
    x1 = base_v2(x1_norm)
    x1 = layers.GlobalAveragePooling2D()(x1)
    
    # --- Branch 2: MobileNetV3-Small (The "Lite" EfficientNet) ---
    # Expects [0, 255] (Handles internal scaling)
    base_v3 = applications.MobileNetV3Small(input_shape=(224,224,3), include_top=False, weights='imagenet')
    base_v3._name = "mobilenet_v3"
    base_v3.trainable = False
    x2 = base_v3(inputs) # Pass raw inputs
    x2 = layers.GlobalAveragePooling2D()(x2)
    
    # --- Fusion ---
    x = layers.concatenate([x1, x2])
    
    # --- Classifier Head ---
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.4)(x) # Stronger dropout for better generalization
    
    outputs = layers.Dense(len(class_names), activation='softmax', dtype='float32')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_fusion_model()
print("\n🧠 Fusion Model Built.")

# 7️⃣ Training
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(CHECKPOINT_DIR, "best_fusion_raf.keras"),
    save_best_only=True, monitor='val_accuracy', mode='max'
)

# Phase 1: Train Head (20 Epochs)
print("\n🚀 Phase 1: Training Head (20 Epochs)...")
model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=[checkpoint])

# Phase 2: Fine-Tuning
print("\n🔓 Phase 2: Fine-Tuning Base Models...")
for layer in model.layers:
    if "mobilenet" in layer.name:
        layer.trainable = True

# Recompile with very low LR for stability
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

print("🚀 Phase 2: Deep Training (30 Epochs)...")
model.fit(train_ds, validation_data=val_ds, epochs=30, callbacks=[checkpoint])

# 8️⃣ Safe TFLite Export (No Flex Ops)
print("\n⚙️ Exporting Clean Model for Pi...")

# Load best weights
model.load_weights(os.path.join(CHECKPOINT_DIR, "best_fusion_raf.keras"))

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS] # Strict CPU mode

tflite_model = converter.convert()

save_path = os.path.join(CHECKPOINT_DIR, "rafdb_fusion_clean.tflite")
with open(save_path, 'wb') as f:
    f.write(tflite_model)

print(f"\n✅ DONE! Download file: rafdb_fusion_clean.tflite")
print(f"ℹ️ Put this on your Pi. NO normalization code changes needed.")